In [1]:
# In this page called Error rates, we are interesed to look into the error rates of each individuals number of (failed, missing  region counts).

In [3]:
# Finding all the statistics of "Failed & Missing" regions.

# 1. distribution of fail and missing counts in pop.(describe)
# 2. 

In [4]:
# Here we are using filtered vcf files for our analysis.

# path : ccmb@mocha:/data/ccmb/malini/repeats/trio_1000_Genomes/Analysis_AV

In [1]:
# Packages

import gzip
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import numpy as np
from zipfile import ZipFile
from collections import OrderedDict
import matplotlib.pyplot as plt
import gzip
import re
import ast
import os
import json
import plotly.express as px

In [37]:
#
Failed_missing_info = {'File_id':'', 'Super_pop':'', 'Sub_pop':'', 'Trio_id':'', 'Genotyping_missed':0, 'Failed_Genotyped':0}

In [3]:
# # putting the all the trio files in dictionary with trio-id and sample index.

directory1 = '../../../../trio_1000_Genomes/Analysis_KT/Pop_analysis/'

samples = {}

with open(f'{directory1}filtered_vcfs_ids.tsv') as fh, open("../../../1KG_Trio_Samples.tsv") as fh1:
    fh.readline()
    fh1.readline()

    for file in tqdm(fh):
        file = file.strip().split("\t")
        samples[file[0]] = ''

    for line in fh1:
        line = line.strip().split('\t')
        samples[line[2]] = samples[line[3]] = samples[line[4]] = line[-1]
        

1788it [00:00, 1456762.93it/s]


In [97]:
samples

{'HG01683': 'EUR',
 'HG01389': 'AMR',
 'HG01608': 'EUR',
 'HG01630': 'EUR',
 'HG03607': 'SAS',
 'HG02285': 'AMR',
 'HG02764': 'AFR',
 'HG03246': 'AFR',
 'HG01938': 'AMR',
 'NA19775': 'AMR',
 'HG02946': 'AFR',
 'HG03195': 'AFR',
 'HG02943': 'AFR',
 'HG01680': 'EUR',
 'HG03100': 'AFR',
 'HG03258': 'AFR',
 'NA19660': 'AMR',
 'HG01170': 'AMR',
 'NA12275': 'EUR',
 'NA10861': 'EUR',
 'NA19206': 'AFR',
 'HG01073': 'AMR',
 'HG02697': 'SAS',
 'NA19782': 'AMR',
 'HG01079': 'AMR',
 'HG01997': 'AMR',
 'HG02279': 'AMR',
 'HG01066': 'AMR',
 'NA19758': 'AMR',
 'HG00595': 'EAS',
 'NA19185': 'AFR',
 'NA12767': 'EUR',
 'NA19100': 'AFR',
 'HG02121': 'EAS',
 'HG02237': 'EUR',
 'HG01347': 'AMR',
 'HG03033': 'AFR',
 'HG00418': 'EAS',
 'HG03101': 'AFR',
 'NA19107': 'AFR',
 'HG03825': 'SAS',
 'HG03894': 'SAS',
 'HG02687': 'SAS',
 'HG03930': 'SAS',
 'NA19655': 'AMR',
 'NA19662': 'AMR',
 'HG01489': 'AMR',
 'HG02887': 'AFR',
 'HG00553': 'AMR',
 'HG03480': 'AFR',
 'NA19746': 'AMR',
 'HG03616': 'SAS',
 'HG01897': 

In [ ]:
# The below code was ran on the bash to use the multithreading.

directory = '../../../Analysis_AV/filtered_vcfs/'
directory1 = '../../../Analysis_KT/Pop_analysis/'

id  = sys.arg[1]

def filtered_vcf(infile):
    out = open(f'{directory1}/Pop_Missed_Failed_counts.tsv', 'a')
    with gzip.open(f'{directory}{infile}', 'rt') as filtered_vcf:
        Failed_Genotyped = 0
        Genotyping_missed = 0
        for line in filtered_vcf:
            line = line.strip().split("\t")
            file_id = infile.split("_")[0]
            
            if line[0].startswith("#"): continue
            if line[-1].startswith('.') or line[-1].startswith('M'):
            
                if line[-1].startswith("."):
                    Failed_Genotyped += 1
                elif line[-1].startswith('M'):
                    Genotyping_missed += 1
    
        print(file_id, Failed_Genotyped, Genotyping_missed, file = out, sep = '\t')
    
    out.close()
    
filtered_vcf(id)

In [4]:
# file with 1788 file ids with its count of missing and failed regions.

df = pd.read_csv("../../Pop_analysis/Pop_Missed_Failed_counts.tsv", header=None, sep = '\t')

In [17]:
# lets add the pop info to df.

super_pop = []

for ind in df.index:
    file_id = df[0][ind]
    super_pop.append(samples[file_id])

df['super_pop'] = pd.DataFrame(super_pop)
Graph0 = df.groupby(["super_pop"], sort=False).apply(lambda x: x.sort_values([1])).reset_index(drop=True)
Graph0.columns = ['sample_id', 'Failed_regions', 'Missed_regions', 'Super_pop']
Graph1 = df.groupby(["super_pop"], sort=False).apply(lambda x: x.sort_values([2])).reset_index(drop=True)
Graph1.columns = Graph0.columns

In [35]:
# counts of the failed and missing of each individuals.

fig = px.box(Graph0, y='Missed_regions', color='Super_pop', title='Range and distribution of num of missing regions in pops')
fig.update_yaxes(title = 'Number of Missing regions')
fig.show()

In [34]:
# stats counts of pop wise.

for pop_name in ['SAS','EUR','AMR','AFR','EAS']:
    temp = df[df['super_pop'] == pop_name]
    print(temp[2].mean(), temp[2].median()) 

696137.9880478088 695928.0
694562.6875 692267.0
698890.3672456576 696200.0
701142.6898839137 699201.0
704797.5071090048 699690.0


In [36]:
# 

In [ ]:
# Below code is to generate the files for both Missing and Failed.


directory = '../../../Analysis_AV/filtered_vcfs/'
directory1 = '../../../Analysis_KT/Error_rates/'

Failed_Missed_Repeat_Regions = {}

id  = sys.arg[1]

def filtered_vcf(infile):

    with gzip.open(f'{directory}{infile}', 'rt') as filtered_vcf:

        for line in filtered_vcf:
            line = line.strip().split("\t")
            file_id = infile.split("_")[0]
            Start = line[7][line[7].find('START=')+6 :]
            Start = int(Start[:Start.find(';')])
            End = line[7][line[7].find('END=')+4 :]
            End = int(End[:End.find(';')])                
            region = f'{line[0]}:{Start}-{End}'   
            rep_len = int(End) - int(Start)
            
            if line[0].startswith("#"): continue
            if line[-1].startswith('.') or line[-1].startswith('M'):
            
                if line[-1].startswith("."):
                    if region not in Failed_Missed_Repeat_Regions:
                        Failed_Missed_Repeat_Regions[region] = [[0,0], [line[0], line[1], rep_len], []]
                        Failed_Missed_Repeat_Regions[region][0][0] += 1
                        Failed_Missed_Repeat_Regions[region][-1].append()
                    else: Failed_Missed_Repeat_Regions[region][0][0] += 1
                elif line[-1].startswith('M'):
                    
    
        print(file_id, Failed_Genotyped, Genotyping_missed, file = out, sep = '\t')
    
    out.close()
    
filtered_vcf(id)